<a href="https://colab.research.google.com/github/gk0434/Gautam_DTSC_Fall2025/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [4]:
1 #Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [5]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [ ]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [7]:
# --- Q1 Functions Implementation ---

def q1_read_table(html: str) -> pd.DataFrame:
    """
    Return the first table with >= 3 columns from the HTML.
    Uses pandas.read_html, picks a reasonable table, then flattens headers.
    """
    tables = pd.read_html(html)
    if not tables:
        raise ValueError("No tables found on the page.")

    # Pick the first table with at least 3 columns
    for t in tables:
        if t.shape[1] >= 3:
            df = t.copy()
            df = flatten_headers(df)
            return df

    raise ValueError("No suitable table (>= 3 columns) found.")


def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean columns:
    - Standardize column names to: Country, Alpha-2, Alpha-3, Numeric
    - Strip whitespace
    - Alpha-2/Alpha-3 -> UPPERCASE
    - Numeric -> nullable int
    - Drop rows with missing required fields
    """
    # Map messy column headers to our target names
    col_map = {}
    for c in df.columns:
        cl = c.strip().lower()
        if "country" in cl:
            col_map[c] = "Country"
        elif "alpha-2" in cl or "alpha 2" in cl:
            col_map[c] = "Alpha-2"
        elif "alpha-3" in cl or "alpha 3" in cl:
            col_map[c] = "Alpha-3"
        elif "numeric" in cl:
            col_map[c] = "Numeric"

    df = df.rename(columns=col_map)

    # Ensure required columns exist
    required = ["Country", "Alpha-2", "Alpha-3", "Numeric"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns after rename: {missing}")

    # Strip whitespace from all object columns
    for c in df.columns:
        if df[c].dtype == "object":
            df[c] = df[c].astype(str).str.strip()

    # Uppercase codes
    df["Alpha-2"] = df["Alpha-2"].str.upper()
    df["Alpha-3"] = df["Alpha-3"].str.upper()

    # Numeric -> nullable int
    df["Numeric"] = pd.to_numeric(df["Numeric"], errors="coerce").astype("Int64")

    # Drop rows missing key fields
    df = df.dropna(subset=["Country", "Alpha-2", "Alpha-3", "Numeric"]).reset_index(drop=True)

    return df


def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """
    Sort descending by Numeric and return Top-N.
    """
    if "Numeric" not in df.columns:
        raise ValueError("Column 'Numeric' not found for sorting.")
    return df.sort_values("Numeric", ascending=False).head(top)


In [8]:
# --- Q1: Write your answer here ---

url_q1 = "https://www.iban.com/country-codes"

# Fetch HTML
html_q1 = fetch_html(url_q1)

# Read & clean table
df_raw_q1 = q1_read_table(html_q1)
df_clean_q1 = q1_clean(df_raw_q1)

# Save to CSV
df_clean_q1.to_csv("data_q1.csv", index=False)
print("✅ Saved cleaned data to data_q1.csv")

# Show Top-15 sorted by Numeric (descending)
top15_q1 = q1_sort_top(df_clean_q1, top=15)
print("✅ Top-15 countries by Numeric code (descending):")
print(top15_q1)


✅ Saved cleaned data to data_q1.csv
✅ Top-15 countries by Numeric code (descending):
                                               Country Alpha-2 Alpha-3  \
247                                             Zambia      ZM     ZMB   
246                                              Yemen      YE     YEM   
192                                              Samoa      WS     WSM   
244                                  Wallis and Futuna      WF     WLF   
240                 Venezuela (Bolivarian Republic of)      VE     VEN   
238                                         Uzbekistan      UZ     UZB   
237                                            Uruguay      UY     URY   
35                                        Burkina Faso      BF     BFA   
243                              Virgin Islands (U.S.)      VI     VIR   
236                     United States of America (the)      US     USA   
219                       Tanzania, United Republic of      TZ     TZA   
108                        

/tmp/ipython-input-4232854299.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [ ]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [9]:
# Q2 — Write your answer here

# --- Q2: Write your answer here ---

# Step 1: Fetch the HTML
url_q2 = "https://news.ycombinator.com/"
html_q2 = fetch_html(url_q2)

# Step 2: Parse items into a DataFrame
from bs4 import BeautifulSoup
import re
import pandas as pd

soup = BeautifulSoup(html_q2, "lxml")
rows = soup.select("tr.athing")
items = []

for row in rows:
    # Rank
    rank_el = row.select_one("span.rank")
    rank_text = rank_el.get_text(strip=True).replace(".", "") if rank_el else ""

    # Title + Link
    title_el = row.select_one("a.storylink") or row.select_one("a.titlelink")
    title = title_el.get_text(strip=True) if title_el else ""
    link = title_el["href"] if title_el and title_el.has_attr("href") else ""

    # Subtext info
    sub = row.find_next_sibling("tr")
    points, comments, user = "", "", ""
    if sub:
        subtext = sub.select_one("td.subtext")
        if subtext:
            score_el = subtext.select_one("span.score")
            if score_el:
                points = score_el.get_text(strip=True)
            user_el = subtext.select_one("a.hnuser")
            if user_el:
                user = user_el.get_text(strip=True)
            for a in subtext.select("a"):
                txt = a.get_text(strip=True).lower()
                if "comment" in txt:
                    comments = a.get_text(strip=True)
                    break

    items.append({
        "rank": rank_text,
        "title": title,
        "link": link,
        "points": points,
        "comments": comments,
        "user": user
    })

df_q2 = pd.DataFrame(items)

# Step 3: Clean and convert numeric columns
def to_int_safe(val):
    m = re.search(r"\d+", str(val))
    return int(m.group(0)) if m else 0

df_q2["rank"] = df_q2["rank"].apply(to_int_safe)
df_q2["points"] = df_q2["points"].apply(to_int_safe)
df_q2["comments"] = df_q2["comments"].apply(to_int_safe)

for col in ["title", "link", "user"]:
    df_q2[col] = df_q2[col].fillna("").astype(str).str.strip()

# Step 4: Save cleaned data
df_q2.to_csv("data_q2.csv", index=False)
print("✅ Saved cleaned data to data_q2.csv")

# Step 5: Show Top-15 sorted by points (desc)
top15_q2 = df_q2.sort_values("points", ascending=False).head(15)
print("✅ Top-15 stories by points (descending):")
print(top15_q2)



✅ Saved cleaned data to data_q2.csv
✅ Top-15 stories by points (descending):
    rank title link  points  comments          user
10    11                440       169   WaitWaitWha
8      9                304       205         art-w
22    23                262        42     kirschner
23    24                254       141     granzymes
5      6                217        82   birdculture
0      1                188        78     vitalnodo
7      8                184        71  joemasilotti
6      7                143        33     zachlatta
3      4                139        57     lalitkale
1      2                115        26   yehiaabdelm
21    22                112        23      azhenley
19    20                105        46       bookmtn
14    15                101        21   bramadityaw
15    16                 78         0      defmarco
29    30                 72        62        ibobev
